<a href="https://colab.research.google.com/github/Okocha76/DS-Unit-2-Kaggle-Challenge/blob/master/module3/LS_DS10_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

## Define a function to wrangle train, validate, and test sets in the same way. Clean outliers and engineer features.

In [3]:
import numpy as np

def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year',
                       'gps_height','population','amount_tsh']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
    
    # Approximate distance from 'Null Island'
    X['distance'] = ((X['latitude']+10.99846435)**2 + (X['longitude']-19.6071219)**2)**.5

    # Convert to datetime and create year_ month_ & day_recorded
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')

    # Engineer feature: how many years from construction_year to date_recorded
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()

    # region_code & district_code are numeric columns, but should be categorical features,
    # so convert it from a number to a string
    X['region_code'] = X['region_code'].astype(str)
    X['district_code'] = X['district_code'].astype(str)
        
    # quantity & quantity_group are duplicates, so drop one
    X = X.drop(columns='quantity_group')
    
    # source, source_class & source_type are almost identical. 
    # source has higher level of detail.
    X = X.drop(columns=['source_class','source_type'])

    # recorded_by has single value, so drop.
    X = X.drop(columns='recorded_by')

    X = X.drop(columns='id')

    # water_quality & quality_group are almost identical. 
    # water_quality has higher level of detail.
    X = X.drop(columns='quality_group')

    # waterpoint_type & waterpoint_type_group are almost identical. 
    # waterpoint_type has higher level of detail.
    X = X.drop(columns='waterpoint_type_group')

    # payment & payment_type are duplicates, so drop one
    X = X.drop(columns='payment_type')

    # extraction_type, extraction_type_class & extraction_type_group are almost identical. 
    # extraction_type has higher level of detail.
    X = X.drop(columns=['extraction_type_class','extraction_type_group'])

    # installer & funder are almost identical. 
    # funder has higher level of detail.
    X = X.drop(columns='installer')     
    
    # management & management_group are almost identical. 
    # management has higher level of detail.
    X = X.drop(columns='management_group') 

    # region_code & region are almost identical. 
    # region_code has higher level of detail.
    X = X.drop(columns='region') 

    # return the wrangled dataframe
    return X

train = wrangle(train)
test = wrangle(test)

## Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.

In [4]:
pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


In [5]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

In [6]:
%%time

target = 'status_group'

features = train.columns.drop(target)
X_train = train[features]
y_train = train[target]


pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median'), 
    RandomForestClassifier(n_estimators=100, max_depth=22, n_jobs=-1, random_state=42)
)

k = 5
scores = cross_val_score(pipeline, X_train, y_train, cv=k, 
                         scoring='accuracy')
print(f'Accuracy for {k} folds:', scores)

Accuracy for 5 folds: [0.81163202 0.80843363 0.80833333 0.81026936 0.8103216 ]
Wall time: 1min 16s


In [7]:
%%time

target = 'status_group'

features = train.columns.drop(target)
X_train = train[features]
y_train = train[target]

# y_train_pre = train[target]
# # Encode target feature, in order to use encoders like Target Encoder
# encoder = ce.OrdinalEncoder()
# y_train = encoder.fit_transform(y_train_pre)

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median'), 
    RandomForestClassifier(n_estimators=100, max_depth=22, n_jobs=-1, random_state=42)
)

param_distributions = {
#     'targetencoder__min_samples_leaf': randint(1, 15), 
#     'targetencoder__smoothing': uniform(1, 50), 
#     'simpleimputer__strategy': ['mean', 'median'], 
#     'randomforestclassifier__n_estimators': randint(80, 120), 
#     'randomforestclassifier__max_depth': range(16, 24), 
    'randomforestclassifier__max_features': uniform(0.2, 0.8),
    'randomforestclassifier__criterion': ['gini', 'entropy'],
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=5, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 15.7min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 21.7min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 24.4min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 28.1min finished


Wall time: 28min 29s


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            f

In [8]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation Accuracy', search.best_score_)

Best hyperparameters {'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__max_features': 0.2627818360633932}
Cross-validation Accuracy 0.8103030303030303


In [9]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,8,5,2,7,1,6,3,9,4,0
mean_fit_time,63.0904,77.0641,151.898,72.9524,176.675,178.11,125.963,114.823,152.877,127.793
std_fit_time,1.94108,2.52371,5.85351,3.11563,2.39735,5.17195,4.4604,15.5145,6.04808,13.8858
mean_score_time,2.48226,2.81682,2.95477,2.84594,2.93351,3.18646,2.0713,2.08476,2.4646,2.41744
std_score_time,0.784206,0.549073,0.950202,1.00094,0.441638,0.729286,0.993191,1.26128,1.18393,1.03182
param_randomforestclassifier__criterion,entropy,gini,entropy,gini,entropy,entropy,gini,gini,gini,gini
param_randomforestclassifier__max_features,0.262782,0.412599,0.590729,0.442942,0.772763,0.77394,0.677832,0.713624,0.850015,0.898479
params,{'randomforestclassifier__criterion': 'entropy...,"{'randomforestclassifier__criterion': 'gini', ...",{'randomforestclassifier__criterion': 'entropy...,"{'randomforestclassifier__criterion': 'gini', ...",{'randomforestclassifier__criterion': 'entropy...,{'randomforestclassifier__criterion': 'entropy...,"{'randomforestclassifier__criterion': 'gini', ...","{'randomforestclassifier__criterion': 'gini', ...","{'randomforestclassifier__criterion': 'gini', ...","{'randomforestclassifier__criterion': 'gini', ..."
split0_test_score,0.813484,0.809191,0.80978,0.809696,0.808349,0.808349,0.806245,0.807424,0.806077,0.801784
split1_test_score,0.809696,0.80776,0.809359,0.807929,0.807087,0.807087,0.806414,0.805656,0.802205,0.802542
split2_test_score,0.808249,0.808586,0.807155,0.808418,0.80564,0.80564,0.807071,0.80564,0.806061,0.804461


## Submit your predictions to DS10 Kaggle competition.

In [10]:
pipeline = search.best_estimator_

In [11]:
y_pred = pipeline.predict(test)

# Makes a dataframe with two columns, id and status_group, 
# and writes to a csv file, without the index

# sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('your-submission-filename.csv', index=False)

In [12]:
# from google.colab import files
# files.download('your-submission-filename.csv')